In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Looking into the calendar
cal = pd.read_csv('./Data/Static_GTFS_MTS_SD/calendar.txt')
cal.head()
# The calendar for the static GTFS data is for 01/29/2017 to 06/10/2017
# It's Active everyday of the week.

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,11112,1,1,1,1,1,1,1,20170129,20170610


In [5]:
cal_date = pd.read_csv('./Data/Static_GTFS_MTS_SD/calendar_dates.txt')
cal_date.head()

,service_id,date,exception_type,exception_note,schedule_type
0,63895,20170129,1,FINAL,SU
1,63921,20170129,1,FINAL,SU
2,64034,20170129,1,PRODUCTION,SU
3,63955,20170129,1,Schedule #105,SU
4,63917,20170130,1,FINAL,WD


In [12]:
stop_times = pd.read_csv('./Data/Static_GTFS_MTS_SD/stop_times.txt')
unique_ids = stop_times['trip_id'].unique()

/Users/AlexanderBaker/anaconda/envs/SD_Transit/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
print(type(unique_ids))
print(len(unique_ids))

<class 'numpy.ndarray'>
20247


In [35]:
from protobuf_to_dict import protobuf_to_dict
import gtfs_rt_pb2 as gtfs_rt
import pandas as pd
from pandas.io.json import json_normalize
import requests

# First script written only works with files.
# f = open('MTS (1).pb', 'rb')
# raw_str = f.read()
#
# msg = gtfs_rt.FeedMessage()
# msg.ParseFromString(raw_str)
#
# # currently working on processing the gtfs rt feed into the correct format.
# print(msg)

feed = gtfs_rt.FeedMessage()
response = requests.get('https://realtime.sdmts.com/api/api/gtfs_realtime/vehicle-positions-for-agency/MTS.pb?key=d6d8bc59-5bd6-4160-acea-2965f2cb48b0')
feed.ParseFromString(response.content)

feed_dict = protobuf_to_dict(feed)
"""
    This is a test run for vehicle position only 
    
    {'id': '1', 
    'vehicle': {
        'trip': {'trip_id': '11975233', 'route_id': '41'}, 
        'timestamp': 1493839585, 
        'vehicle': {'id': '911'}, 
        'position': {'latitude': 32.86729049682617, 'longitude': -117.21359252929688}
        }
    }
    
    python

    
"""
# print(sorted(feed_dict['entity'].iteritems(), key=lambda (k,v): (v,k)))

prep_dict_lst = []


for entity in feed_dict['entity']:
    
    sort = sorted(entity['vehicle'].items())
    sorted_dict = {}
    
    for key, val in sort:
        sorted_dict[key] = val
    
#     prep_dict_lst.append(sorted_dict)
    break

# print(prep_dict_lst)

In [36]:
from pymongo import MongoClient

In [37]:
client = MongoClient('mongodb://Alexanders-MacBook-Pro-2.local/testing')
db = client.testing
vehiclePos = db.vehiclePosition

In [38]:
"""
Figured out the problem. I wasn't properly referencing the database.
However, only currently calling the current one. Not able to interact with mongodb accross the internet yet.
"""
result = vehiclePos.insert_many(feed_dict['entity'])

ServerSelectionTimeoutError: alexanders-macbook-pro-2.local:27017: [Errno 8] nodename nor servname provided, or not known

In [ ]:
for pos in vehiclePos.find():
    pprint.pprint(post)